### Mixing model

Test different model

In [ ]:
import os, sys
import numpy as np
from matplotlib import pyplot as plt

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))

import shilofue.Rheology as Rheology

# import utilities in subdirectiory
sys.path.append(os.path.join(ASPECT_LAB_DIR, 'utilities', "python_scripts"))
import Utilities

In [ ]:
def IsoStrainMix(viscosities, v_fractions):
    '''
    iso strain mixing model
    inputs:
        etas: values of viscosities
        v_fractions: volume fractions
    '''
    assert(len(viscosities) == len(v_fractions))

    assemblage_viscosity = 0.0
    for i in range(len(viscosities)):
        viscosity = viscosities[i]
        v_fraction = v_fractions[i]
        assemblage_viscosity += v_fraction*viscosity

    return assemblage_viscosity


def IsoStressMix(viscosities, v_fractions):
    '''
    iso stress mixing model
    inputs:
        etas: values of viscosities
        v_fractions: volume fractions
    '''
    assert(len(viscosities) == len(v_fractions))

    temp = 0.0
    for i in range(len(viscosities)):
        viscosity = viscosities[i]
        v_fraction = v_fractions[i]
        temp += v_fraction / viscosity
    assemblage_viscosity = 1.0 / temp

    return assemblage_viscosity


def LogMix(viscosities, v_fractions):
    '''
    logrithm mixing model
    inputs:
        etas: values of viscosities
        v_fractions: volume fractions
    '''
    assert(len(viscosities) == len(v_fractions))

    temp = 0.0
    for i in range(len(viscosities)):
        log_viscosity = np.log10(viscosities[i])
        v_fraction = v_fractions[i]
        temp += v_fraction * log_viscosity
    assemblage_viscosity = 10**temp

    return assemblage_viscosity


def TullisMix(creeps, v_fractions, strain_rate, P, T, d, Coh):
    '''
    '''
    assert(len(creeps) == 2)
    assemblage_creep = creeps[0].copy()

    temp = 0.0
    for i in range(len(creeps)):
        temp += v_fractions[i] * np.log10(creeps[i]['n'])
    n = 10**temp

    E = (creeps[1]['E'] * (n - creeps[0]['n']) - creeps[0]['E'] * (n - creeps[1]['n'])) / (creeps[1]['n'] - creeps[0]['n'])

    temp1 = 0.0
    temp1 = (np.log10(creeps[1]['A']) * (n - creeps[0]['n']) - np.log10(creeps[0]['A']) * (n - creeps[1]['n'])) / (creeps[1]['n'] - creeps[0]['n'])
    A = 10**temp1
    
    assemblage_creep['n'] = n
    assemblage_creep['A'] = A
    assemblage_creep['E'] = E

    assemblage_viscosity = Rheology.CreepRheology(assemblage_creep, strain_rate, P, T, d, Coh)

    return assemblage_viscosity

In [ ]:
RheologyPrm = Rheology.RHEOLOGY_PRM()
diffusion_creep = RheologyPrm.HK03_diff
dislocation_creep = RheologyPrm.HK03_disl

dislocation_creep_1 = dislocation_creep.copy()
dislocation_creep_2 = dislocation_creep.copy()

dislocation_creep_2['A'] /= 10.0
dislocation_creep_2['E'] /= 2.0
dislocation_creep_2['V'] /= 2.0
dislocation_creep_2['n'] /= 2.0

strain_rate = 2.5e-12
P = 1e9
T = 1400.0 + 273.15
d = 1e4
Coh = 1000.0
eta1 = Rheology.CreepRheology(dislocation_creep_1, strain_rate, P, T, d, Coh)
eta2 = Rheology.CreepRheology(dislocation_creep_2, strain_rate, P, T, d, Coh)

print("eta1 = ", eta1)
print("eta2 = ", eta2)

# eta1 = 1e24
# eta2 = 1e20

n_s = 100
v0s = np.linspace(0.0, 1.0, n_s)
iso_strain_viscosities = np.zeros(n_s)
iso_stress_viscosities = np.zeros(n_s)
log_mixing_viscosities = np.zeros(n_s)
tullis_mixing_viscosities = np.zeros(n_s)

for i in range(n_s):
    v0 = v0s[i]
    v_fractions = [v0, 1-v0]
    iso_strain_viscosities[i] = IsoStrainMix([eta1, eta2], v_fractions)
    iso_stress_viscosities[i] = IsoStressMix([eta1, eta2], v_fractions)
    log_mixing_viscosities[i] = LogMix([eta1, eta2], v_fractions)
    tullis_mixing_viscosities[i] = TullisMix([dislocation_creep_1, dislocation_creep_2], v_fractions, strain_rate, P, T, d, Coh)

fig, ax = plt.subplots()
ax.semilogy(v0s, iso_strain_viscosities, label='iso strain')
ax.semilogy(v0s, iso_stress_viscosities, label='iso stress')
ax.semilogy(v0s, log_mixing_viscosities, label='log')
ax.semilogy(v0s, tullis_mixing_viscosities, label='tullis')
ax.set_xlabel('volume fraction')
ax.set_ylabel('viscosity (Pa*s)')
ax.legend()

fig.savefig(os.path.join(RESULT_DIR, "mixing_model.pdf"))